In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from scipy.optimize import minimize

# 1. Prepare the Data (Extracted from your files for Function 5)
# Inputs (X)
X_train = np.array([
    [0.3, 0.3, 0.3, 0.3],                       # Week 1
    [0.28, 0.32, 0.3, 0.29],                    # Week 2
    [0.344822, 0.264687, 0.374156, 0.203902]    # Week 3
])

# Outputs (y)
y_train = np.array([
    136.85104415360001,
    137.29069123056925,
    131.78253310559094
])

# 2. Define the Kernel
# We use an RBF kernel (standard for smooth functions) + WhiteKernel (to handle noise/uncertainty)
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=np.ones(4), length_scale_bounds=(1e-2, 1e2)) + \
         WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1))

# 3. Fit the Gaussian Process
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True, random_state=42)
gp.fit(X_train, y_train)

# 4. Define the Acquisition Function
# We want to find x that maximizes the GP's predicted mean.
# Since scipy.optimize minimizes, we return the negative predicted value.
def surrogate_function(x):
    x = x.reshape(1, -1)
    # We predict the mean. 
    # Note: In a true exploration scenario, we might use Upper Confidence Bound (UCB),
    # but for a "best guess" submission, maximizing the mean is often safest.
    prediction, _ = gp.predict(x, return_std=True)
    return -prediction[0] # Negative because we want to maximize

# 5. Optimize to find the best next input
# We start the search from our current best point (Week 2)
x0 = X_train[1] 

# Constraints: Inputs must be positive.
# We bound the search reasonably around our data to avoid wild extrapolation.
bounds = [(0.001, 1.0) for _ in range(4)]

res = minimize(
    surrogate_function, 
    x0, 
    bounds=bounds, 
    method='L-BFGS-B'
)

# 6. Display Results
print("Model details:", gp.kernel_)
print(f"Predicted Best Input for Week 4: {res.x}")
print(f"Predicted Output: {-res.fun}")

# Formatting for easy copy-paste
formatted_input = ", ".join([f"{val:.6f}" for val in res.x])
print(f"\nSubmit this array:\n[{formatted_input}]")

Model details: 1.11**2 * RBF(length_scale=[0.128, 77.9, 0.01, 34]) + WhiteKernel(noise_level=0.000345)
Predicted Best Input for Week 4: [0.19682784 0.32001699 0.29999999 0.28995751]
Predicted Output: 138.20329846542452

Submit this array:
[0.196828, 0.320017, 0.300000, 0.289958]


C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
